In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from holoviews.plotting.bokeh.styles import alpha
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier


import warnings
warnings.filterwarnings('ignore')


In [10]:
# importing csv
df = pd.read_csv('extractedMimic.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4559 entries, 0 to 4558
Columns: 106 entries, icustay_id to crystalloid_bolus
dtypes: float64(57), int64(39), object(10)
memory usage: 3.7+ MB


In [4]:
df.drop(df[df['age'] < 18].index, inplace=True)

df.describe()

,icustay_id,hadm_id,suspected_infection_time_poe_days,positiveculture_poe,blood_culture_positive,age,is_male,race_white,race_black,race_hispanic,...,glucose_min1,glucose_max1,glucose_mean,rrt,subject_id,hadm_id.1,icustay_id.1,urineoutput,colloid_bolus,crystalloid_bolus
count,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,...,4525.000000,4525.000000,4525.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,508.000000,3361.000000
mean,250637.474863,149922.364874,0.084101,0.136334,0.381339,65.173619,0.565971,0.718771,0.086279,0.032931,...,111.232044,411.037348,177.261026,0.043469,68174.064105,149922.364874,250637.474863,1842.618441,382.694882,645.671229
std,28759.579721,28710.589841,0.290748,0.343180,0.485769,17.622075,0.495683,0.449649,0.280806,0.178475,...,36.773707,14863.345644,2123.773411,0.203932,18470.812604,28710.589841,28759.579721,1535.550605,134.934798,370.024064
min,200075.000000,100003.000000,-0.991076,0.000000,0.000000,18.020900,0.000000,0.000000,0.000000,0.000000,...,12.000000,57.000000,52.444444,0.000000,165.000000,100003.000000,200075.000000,0.000000,150.000000,250.000000
25%,225575.500000,125404.500000,-0.075677,0.000000,0.000000,53.784850,0.000000,0.000000,0.000000,0.000000,...,89.000000,130.000000,112.888889,0.000000,53134.000000,125404.500000,225575.500000,897.500000,250.000000,500.000000
50%,250984.000000,149667.000000,0.034965,0.000000,0.000000,66.591500,1.000000,1.000000,0.000000,0.000000,...,106.000000,166.000000,134.000000,0.000000,68391.000000,149667.000000,250984.000000,1560.000000,500.000000,500.000000
75%,275436.000000,175042.500000,0.157309,0.000000,1.000000,79.540400,1.000000,1.000000,0.000000,0.000000,...,127.000000,217.000000,165.000000,0.000000,83771.500000,175042.500000,275436.000000,2460.000000,500.000000,1000.000000
max,299998.000000,199962.000000,0.995139,1.000000,1.000000,91.400000,1.000000,1.000000,1.000000,1.000000,...,480.000000,999999.000000,142966.857100,1.000000,99982.000000,199962.000000,299998.000000,50515.000000,1000.000000,11000.000000


In [5]:
X = df[['urineoutput', 'lactate_min','bun_mean','sysbp_min', 'metastatic_cancer', 'inr_max', 'age', 'sodium_max', 'aniongap_max', 'creatinine_min', 'spo2_mean']]

y = df['thirtyday_expire_flag']

In [6]:
X.fillna(X.median(), inplace=True)

In [7]:
X.isnull().sum()

urineoutput          0
lactate_min          0
bun_mean             0
sysbp_min            0
metastatic_cancer    0
inr_max              0
age                  0
sodium_max           0
aniongap_max         0
creatinine_min       0
spo2_mean            0
dtype: int64

In [8]:
X.shape

(4555, 11)

In [9]:
# Step 3: Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (3644, 11)
Testing set shape: (911, 11)


# -----------------------------------------------------------------------

# Testing Stacking

In [43]:


# Step 1: Define the parameter grid for hyperparameter tuning
param_grid = {
    # 'n_estimators': [100],
    # 'max_depth': [6],
    # 'learning_rate': [0.01, 0.1],
    # 'subsample': [1.0],
    # 'gamma': [0, 0.1, 0.2],  # Regularization parameter for tree pruning
#     'alpha': [0, 0.1, 0.2],  # L1 regularization term
#     'lambda': [1, 1.5, 2]    # L2 regularization term
    'n_estimators': [100],      # Increased number of trees for better learning
    'max_depth': [6],           # Control the depth of trees
    'learning_rate': [0.05],    # Reduced learning rate
    'subsample': [0.8],         # Control overfitting by using 80% of data for each tree
    'colsample_bytree': [0.8]   # Featu
}

# Step 2: Define base models for stacking
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', booster='gbtree', objective='binary:logistic')),
    ('lr', LogisticRegression(max_iter=1000))
]

# Step 3: Define the stacking classifier with a logistic regression as the final estimator
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Step 4: Set up the GridSearchCV
grid_search = GridSearchCV(estimator=stacking_model, param_grid={'xgb__' + k: v for k, v in param_grid.items()}, scoring='roc_auc', cv=5, verbose=1)

# Step 5: Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
best_stacking_model = grid_search.best_estimator_

# Make predictions on the test set
y_probs = best_stacking_model.predict_proba(X_test)[:, 1]  # Predicted probabilities for class 1

# Step 7: Evaluate the model's performance with the default threshold of 0.8
default_threshold = 0.8  # Set the default threshold to 0.8
y_pred_default = (y_probs >= default_threshold).astype(int)
accuracy_default = accuracy_score(y_test, y_pred_default)
auc_default = roc_auc_score(y_test, y_probs)
conf_matrix_default = confusion_matrix(y_test, y_pred_default)
class_report_default = classification_report(y_test, y_pred_default)

# Print evaluation metrics for the default threshold
print(f"Default Threshold ({default_threshold:.1f}):")
print(f"Model Accuracy: {accuracy_default * 100:.5f}%")
print(f"Model AUC: {auc_default * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_default)
print("Classification Report:")
print(class_report_default)

# Step 8: Threshold tuning
# Determine the best threshold
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

# Step 9: Evaluate the model's performance with the best threshold
y_pred_best = (y_probs >= best_threshold).astype(int)
accuracy_best = accuracy_score(y_test, y_pred_best)
auc_best = roc_auc_score(y_test, y_probs)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
class_report_best = classification_report(y_test, y_pred_best)

# Print evaluation metrics for the best threshold
print(f"\nBest Threshold: {best_threshold:.2f}")
print("Best Threshold Performance:")
print(f"Model Accuracy: {accuracy_best * 100:.5f}%")
print(f"Model AUC: {auc_best * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_best)
print("Classification Report:")
print(class_report_best)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Default Threshold (0.8):
Model Accuracy: 83.97366%
Model AUC: 84.09910%
Confusion Matrix:
[[727   2]
 [144  38]]
Classification Report:
              precision    recall  f1-score   support

           0       0.83      1.00      0.91       729
           1       0.95      0.21      0.34       182

    accuracy                           0.84       911
   macro avg       0.89      0.60      0.63       911
weighted avg       0.86      0.84      0.80       911


Best Threshold: 0.21
Best Threshold Performance:
Model Accuracy: 80.79034%
Model AUC: 84.09910%
Confusion Matrix:
[[612 117]
 [ 58 124]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.87       729
           1       0.51      0.68      0.59       182

    accuracy                           0.81       911
   macro avg       0.71      0.76      0.73       911
weighted avg       0.83      0.81      0.82  

In [13]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score

# Step 1: Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100],
    'max_depth': [3],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],  # Regularization parameter for tree pruning
    'colsample_bytree': [0.8],

}

# Step 2: Define base models for stacking
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', booster='gbtree', objective='binary:logistic')),
    ('lr', LogisticRegression(max_iter=1000))
]

# Step 3: Define the stacking classifier with logistic regression as the final estimator
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Step 4: Set up the GridSearchCV
grid_search = GridSearchCV(estimator=stacking_model, param_grid={'xgb__' + k: v for k, v in param_grid.items()}, 
                           scoring='roc_auc', cv=5, verbose=1)

# Step 5: Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
best_stacking_model = grid_search.best_estimator_

# Step 6: Make predictions on the test set
y_probs = best_stacking_model.predict_proba(X_test)[:, 1]  # Predicted probabilities for class 1

# Step 7: Evaluate the model's performance with the default threshold of 0.5
y_pred_default = (y_probs >= 0.5).astype(int)
accuracy_default = accuracy_score(y_test, y_pred_default)
auc_default = roc_auc_score(y_test, y_probs)
conf_matrix_default = confusion_matrix(y_test, y_pred_default)
class_report_default = classification_report(y_test, y_pred_default)

# Print evaluation metrics for the default threshold
print("Default Threshold (0.5):")
print(f"Model Accuracy: {accuracy_default * 100:.5f}%")
print(f"Model AUC: {auc_default * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_default)
print("Classification Report:")
print(class_report_default)

# Step 8: Threshold tuning
# Determine the best threshold
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

# Step 9: Evaluate the model's performance with the best threshold
y_pred_best = (y_probs >= best_threshold).astype(int)
accuracy_best = accuracy_score(y_test, y_pred_best)
auc_best = roc_auc_score(y_test, y_probs)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
class_report_best = classification_report(y_test, y_pred_best)

# Print evaluation metrics for the best threshold
print(f"\nBest Threshold: {best_threshold:.2f}")
print("Best Threshold Performance:")
print(f"Model Accuracy: {accuracy_best * 100:.5f}%")
print(f"Model AUC: {auc_best * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_best)
print("Classification Report:")
print(class_report_best)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Default Threshold (0.5):
Model Accuracy: 85.40066%
Model AUC: 83.84434%
Confusion Matrix:
[[707  22]
 [111  71]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       729
           1       0.76      0.39      0.52       182

    accuracy                           0.85       911
   macro avg       0.81      0.68      0.72       911
weighted avg       0.84      0.85      0.83       911


Best Threshold: 0.22
Best Threshold Performance:
Model Accuracy: 81.11965%
Model AUC: 83.84434%
Confusion Matrix:
[[616 113]
 [ 59 123]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.84      0.88       729
           1       0.52      0.68      0.59       182

    accuracy                           0.81       911
   macro avg       0.72      0.76      0.73       911
weighted avg       0.83      0.81      0.82

In [29]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score

# Step 1: Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100],
    'max_depth': [3],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],  # Regularization parameter for tree pruning
    'colsample_bytree': [0.8],
}

# Step 2: Define base models for stacking
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', booster='gbtree', objective='binary:logistic')),
    ('lr', LogisticRegression(max_iter=1000))
]

# Step 3: Define the stacking classifier with logistic regression as the final estimator
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Step 4: Set up the GridSearchCV
grid_search = GridSearchCV(estimator=stacking_model, param_grid={'xgb__' + k: v for k, v in param_grid.items()}, 
                           scoring='roc_auc', cv=5, verbose=1)

# Step 5: Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
best_stacking_model = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:")
print(grid_search.best_params_)

# Step 6: Make predictions on the test set
y_probs = best_stacking_model.predict_proba(X_test)[:, 1]  # Predicted probabilities for class 1

# Step 7: Evaluate the model's performance with the default threshold of 0.5
y_pred_default = (y_probs >= 0.5).astype(int)
accuracy_default = accuracy_score(y_test, y_pred_default)
auc_default = roc_auc_score(y_test, y_probs)
conf_matrix_default = confusion_matrix(y_test, y_pred_default)
class_report_default = classification_report(y_test, y_pred_default)

# Print evaluation metrics for the default threshold
print("Default Threshold (0.5):")
print(f"Model Accuracy: {accuracy_default * 100:.5f}%")
print(f"Model AUC: {auc_default * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_default)
print("Classification Report:")
print(class_report_default)

# Step 8: Threshold tuning
# Determine the best threshold
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

# Step 9: Evaluate the model's performance with the best threshold
y_pred_best = (y_probs >= best_threshold).astype(int)
accuracy_best = accuracy_score(y_test, y_pred_best)
auc_best = roc_auc_score(y_test, y_probs)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
class_report_best = classification_report(y_test, y_pred_best)

# Print evaluation metrics for the best threshold
print(f"\nBest Threshold: {best_threshold:.2f}")
print("Best Threshold Performance:")
print(f"Model Accuracy: {accuracy_best * 100:.5f}%")
print(f"Model AUC: {auc_best * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_best)
print("Classification Report:")
print(class_report_best)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters:
{'xgb__colsample_bytree': 0.8, 'xgb__gamma': 0.1, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 100, 'xgb__subsample': 0.8}
Default Threshold (0.5):
Model Accuracy: 85.94951%
Model AUC: 84.25361%
Confusion Matrix:
[[710  19]
 [109  73]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       729
           1       0.79      0.40      0.53       182

    accuracy                           0.86       911
   macro avg       0.83      0.69      0.73       911
weighted avg       0.85      0.86      0.84       911


Best Threshold: 0.19
Best Threshold Performance:
Model Accuracy: 79.69265%
Model AUC: 84.25361%
Confusion Matrix:
[[596 133]
 [ 52 130]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.82      0.87       729
           1       0.49      0.71      0.5

In [14]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score

# Step 1: Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100],
    'max_depth': [3],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],  # Regularization parameter for tree pruning
    'colsample_bytree': [0.8],

}

# Step 2: Define base models for stacking
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', booster='gbtree', objective='binary:logistic')),
    ('lr', LogisticRegression(max_iter=1000))
]

# Step 3: Define the stacking classifier with logistic regression as the final estimator
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Step 4: Set up the GridSearchCV
grid_search = GridSearchCV(estimator=stacking_model, param_grid={'xgb__' + k: v for k, v in param_grid.items()}, 
                           scoring='roc_auc', cv=9, verbose=1)

# Step 5: Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
best_stacking_model = grid_search.best_estimator_

# Step 6: Make predictions on the test set
y_probs = best_stacking_model.predict_proba(X_test)[:, 1]  # Predicted probabilities for class 1

# Step 7: Evaluate the model's performance with the default threshold of 0.5
y_pred_default = (y_probs >= 0.5).astype(int)
accuracy_default = accuracy_score(y_test, y_pred_default)
auc_default = roc_auc_score(y_test, y_probs)
conf_matrix_default = confusion_matrix(y_test, y_pred_default)
class_report_default = classification_report(y_test, y_pred_default)

# Print evaluation metrics for the default threshold
print("Default Threshold (0.5):")
print(f"Model Accuracy: {accuracy_default * 100:.5f}%")
print(f"Model AUC: {auc_default * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_default)
print("Classification Report:")
print(class_report_default)

# Step 8: Threshold tuning
# Determine the best threshold
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

# Step 9: Evaluate the model's performance with the best threshold
y_pred_best = (y_probs >= best_threshold).astype(int)
accuracy_best = accuracy_score(y_test, y_pred_best)
auc_best = roc_auc_score(y_test, y_probs)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
class_report_best = classification_report(y_test, y_pred_best)

# Print evaluation metrics for the best threshold
print(f"\nBest Threshold: {best_threshold:.2f}")
print("Best Threshold Performance:")
print(f"Model Accuracy: {accuracy_best * 100:.5f}%")
print(f"Model AUC: {auc_best * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_best)
print("Classification Report:")
print(class_report_best)


Fitting 9 folds for each of 12 candidates, totalling 108 fits
Default Threshold (0.5):
Model Accuracy: 85.18112%
Model AUC: 84.10136%
Confusion Matrix:
[[706  23]
 [112  70]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       729
           1       0.75      0.38      0.51       182

    accuracy                           0.85       911
   macro avg       0.81      0.68      0.71       911
weighted avg       0.84      0.85      0.83       911


Best Threshold: 0.21
Best Threshold Performance:
Model Accuracy: 80.57080%
Model AUC: 84.10136%
Confusion Matrix:
[[607 122]
 [ 55 127]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.83      0.87       729
           1       0.51      0.70      0.59       182

    accuracy                           0.81       911
   macro avg       0.71      0.77      0.73       911
weighted avg       0.84      0.81      0.8

In [32]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, f1_score

# Step 1: Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100],
    'max_depth': [3],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],  # Regularization parameter for tree pruning
    'colsample_bytree': [0.8],

}

# Step 2: Define base models for stacking
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', booster='gbtree', objective='binary:logistic')),
    ('lr', LogisticRegression(max_iter=1000))
]

# Step 3: Define the stacking classifier with logistic regression as the final estimator
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Step 4: Set up the GridSearchCV
grid_search = GridSearchCV(estimator=stacking_model, param_grid={'xgb__' + k: v for k, v in param_grid.items()}, 
                           scoring='roc_auc', cv=12, verbose=1)

# Step 5: Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
best_stacking_model = grid_search.best_estimator_

# Step 6: Make predictions on the test set
y_probs = best_stacking_model.predict_proba(X_test)[:, 1]  # Predicted probabilities for class 1

# Step 7: Evaluate the model's performance with the default threshold of 0.5
y_pred_default = (y_probs >= 0.5).astype(int)
accuracy_default = accuracy_score(y_test, y_pred_default)
auc_default = roc_auc_score(y_test, y_probs)
conf_matrix_default = confusion_matrix(y_test, y_pred_default)
class_report_default = classification_report(y_test, y_pred_default)

# Print evaluation metrics for the default threshold
print("Default Threshold (0.5):")
print(f"Model Accuracy: {accuracy_default * 100:.5f}%")
print(f"Model AUC: {auc_default * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_default)
print("Classification Report:")
print(class_report_default)

# Step 8: Threshold tuning
# Determine the best threshold
thresholds = np.arange(0.0, 1.0, 0.01)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    y_pred = (y_probs >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

# Step 9: Evaluate the model's performance with the best threshold
y_pred_best = (y_probs >= best_threshold).astype(int)
accuracy_best = accuracy_score(y_test, y_pred_best)
auc_best = roc_auc_score(y_test, y_probs)
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
class_report_best = classification_report(y_test, y_pred_best)

# Print evaluation metrics for the best threshold
print(f"\nBest Threshold: {best_threshold:.2f}")
print("Best Threshold Performance:")
print(f"Model Accuracy: {accuracy_best * 100:.5f}%")
print(f"Model AUC: {auc_best * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix_best)
print("Classification Report:")
print(class_report_best)


Fitting 12 folds for each of 12 candidates, totalling 144 fits
Default Threshold (0.5):
Model Accuracy: 85.62020%
Model AUC: 84.30863%
Confusion Matrix:
[[710  19]
 [112  70]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.92       729
           1       0.79      0.38      0.52       182

    accuracy                           0.86       911
   macro avg       0.83      0.68      0.72       911
weighted avg       0.85      0.86      0.84       911


Best Threshold: 0.22
Best Threshold Performance:
Model Accuracy: 81.22942%
Model AUC: 84.30863%
Confusion Matrix:
[[614 115]
 [ 56 126]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.84      0.88       729
           1       0.52      0.69      0.60       182

    accuracy                           0.81       911
   macro avg       0.72      0.77      0.74       911
weighted avg       0.84      0.81      0.

In [40]:
# Step 1: Train the XGBoost model with hyperparameter tuning
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False, 
    eval_metric='logloss', 
    booster='gbtree', 
    objective='binary:logistic', 
    n_estimators=90,      # Increased number of trees for better learning
    max_depth=6,           # Control the depth of trees
    learning_rate=0.05,    # Reduced learning rate
    subsample=0.8,         # Control overfitting by using 80% of data for each tree
    colsample_bytree=0.8,   # Feature subsampling
    gamma = 0.05,
)
xgb_model.fit(X_train, y_train)

# Step 2: Make predictions on the test set
y_pred = xgb_model.predict(X_test)  # Predicted labels (0 or 1)
y_probs = xgb_model.predict_proba(X_test)[:, 1]  # Predicted probabilities for class 1

# Step 3: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)  # Accuracy based on predicted labels
auc = roc_auc_score(y_test, y_probs)  # AUC based on predicted probabilities
conf_matrix = confusion_matrix(y_test, y_pred)  # Confusion matrix based on predicted labels
class_report = classification_report(y_test, y_pred)  # Classification report based on predicted labels

# Print evaluation metrics
print(f"Model Accuracy: {accuracy * 100:.5f}%")
print(f"Model AUC: {auc * 100:.5f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Model Accuracy: 85.51043%
Model AUC: 84.74804%
Confusion Matrix:
[[712  17]
 [115  67]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92       729
           1       0.80      0.37      0.50       182

    accuracy                           0.86       911
   macro avg       0.83      0.67      0.71       911
weighted avg       0.85      0.86      0.83       911

